In [16]:
#import langchain csv loader
from langchain.document_loaders import CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import pandas as pd
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain_community.vectorstores.utils import DistanceStrategy

from transformers import AutoTokenizer, pipeline, AutoModelForCausalLM, GenerationConfig
from langchain import HuggingFacePipeline
from langchain.chains import RetrievalQA
import torch

### <center>  **`Embeddings`**

In [8]:
loader = CSVLoader('../data/VALERIE_HAYER.csv', metadata_columns=["source", "source_date"], encoding="utf-8")
documents = loader.load()

In [9]:
# from sentence_transformers import SentenceTransformer
# from tqdm import tqdm
# import matplotlib.pyplot as plt

#  # To get the value of the max sequence_length, we will query the underlying `SentenceTransformer` object used in the RecursiveCharacterTextSplitter.
# print(f"Model's maximum sequence length: {SentenceTransformer('thenlper/gte-small').max_seq_length}")

# from transformers import AutoTokenizer
# tokenizer = AutoTokenizer.from_pretrained("thenlper/gte-small")
# lengths = [len(tokenizer.encode(doc.page_content)) for doc in tqdm(chunked_docs)]

# # Plot the distrubution of document lengths, counted as the number of tokens
# fig = pd.Series(lengths).hist()
# plt.title("Distribution of document lengths in the knowledge base (in count of tokens)")
# plt.show()

In [10]:
splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=30)
chunked_docs = splitter.split_documents(documents)

In [11]:
EMBEDDING_MODEL_NAME= 'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2'

embedding_model = HuggingFaceEmbeddings(
    model_name=EMBEDDING_MODEL_NAME,
    multi_process=True,
    model_kwargs={"device": "cpu"},
    encode_kwargs={"normalize_embeddings": True},  # set True for cosine similarity
)

In [12]:
KNOWLEDGE_VECTOR_DATABASE = FAISS.from_documents(
    chunked_docs, embedding_model, distance_strategy=DistanceStrategy.COSINE
)

In [13]:
KNOWLEDGE_VECTOR_DATABASE

In [31]:
# file_path = "knowledge_vector_database.faiss"
# # Sauvegarder la base de vecteurs dans le fichier
# FAISS.write_index(KNOWLEDGE_VECTOR_DATABASE, file_path)

In [14]:
# Test 
question = "Que pense Valérie Hayer de la guerre à Gaza ? "
searchDocs = KNOWLEDGE_VECTOR_DATABASE.similarity_search(question)
print(searchDocs[0].page_content)

contenu: J’ai voté en faveur de cette résolution relative aux attaques infâmes menées par le groupe terroriste du Hamas en Israël, ainsi qu’à la détérioration très préoccupante de la situation humanitaire dans la région. Je condamne avec la plus grande fermeté les attaques barbares menées par le Hamas contre Israël, ayant conduit à la mort de plus de 1400 Israéliens et de nombreuses prises d’otage. Face à ces attaques, notre discours doit être ferme, sans pudeur ni complaisance : le Hamas est un groupe


### <center> **`Preparing LLM`**

In [30]:

from transformers import AutoModelForCausalLM, AutoTokenizer
#device = "cuda" # the device to load the model onto
llm_model= "mistralai/Mistral-7B-Instruct-v0.2"
model = AutoModelForCausalLM.from_pretrained(llm_model)
tokenizer = AutoTokenizer.from_pretrained(llm_model)

Loading checkpoint shards: 100%|██████████| 3/3 [01:06<00:00, 22.21s/it]


In [45]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_name = "HuggingFaceH4/zephyr-7b-beta"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config)
tokenizer = AutoTokenizer.from_pretrained(model_name)

ImportError: Using `bitsandbytes` 8-bit quantization requires Accelerate: `pip install accelerate` and the latest version of bitsandbytes: `pip install -i https://pypi.org/simple/ bitsandbytes`

In [38]:
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from transformers import pipeline
from langchain_core.output_parsers import StrOutputParser

text_generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.2,
    do_sample=True,
    repetition_penalty=1.1,
    return_full_text=True,
    max_new_tokens=400,
    pad_token_id=tokenizer.eos_token_id,
)

llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

prompt_template = """
<|system|>
Réponds à la question en utilisant ta connaissance. Utilise le contexte suivant pour t'aider:

{context}

</s>
<|user|>
{question}
</s>
<|assistant|>

 """

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)

llm_chain = prompt | llm | StrOutputParser()

In [39]:
from langchain_core.runnables import RunnablePassthrough

retriever = KNOWLEDGE_VECTOR_DATABASE.as_retriever()

rag_chain = {"context": retriever, "question": RunnablePassthrough()} | llm_chain

In [40]:
question = "Que pense Valérie Hayer de la guerre à Gaza?"

In [41]:
rag_chain.invoke(question)

KeyboardInterrupt: 

In [54]:
pip install accelerate

Note: you may need to restart the kernel to use updated packages.


In [15]:
MODEL_NAME = "mistralai/Mistral-7B-Instruct-v0.1"


tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME, torch_dtype=torch.float16,
    trust_remote_code=True,
    low_cpu_mem_usage=True
)


Loading checkpoint shards: 100%|██████████| 2/2 [00:48<00:00, 24.18s/it]


NameError: name 'GenerationConfig' is not defined

In [20]:

generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
generation_config.max_new_tokens = 1024
generation_config.temperature = 0.0001
generation_config.top_p = 0.95
generation_config.do_sample = True
generation_config.repetition_penalty = 1.15

pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=True,
    generation_config=generation_config,

)

ValueError: The following `model_kwargs` are not used by the model: ['model', 'tokenizer'] (note: typos in the generate arguments will also show up in this list)

In [ ]:
llm = HuggingFacePipeline(
    pipeline=pipeline,
    )

In [21]:
query = "Explain the difference between ChatGPT and open source LLMs in a couple of lines."
result = llm(
    query
)

display(Markdown(f"<b>{query}</b>"))
display(Markdown(f"<p>{result}</p>"))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [48]:
from langchain_community.document_loaders import TextLoader
from langchain_mistralai.chat_models import ChatMistralAI
from langchain_mistralai.embeddings import MistralAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain

loader = CSVLoader('../data/VALERIE_HAYER.csv', metadata_columns=["source", "source_date"], encoding="utf-8")
documents = loader.load()
# Split text into chunks 
text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(documents)
# Define the embedding model
embeddings = MistralAIEmbeddings(model="mistral-embed", mistral_api_key=api_key)
# Create the vector store 
vector = FAISS.from_documents(documents, embeddings)
# Define a retriever interface
retriever = vector.as_retriever()
# Define LLM
model = ChatMistralAI(mistral_api_key=api_key)
# Define prompt template
prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

# Create a retrieval chain to answer questions
document_chain = create_stuff_documents_chain(model, prompt)
retrieval_chain = create_retrieval_chain(retriever, document_chain)
response = retrieval_chain.invoke({"input": "What were the two main things the author worked on before college?"})
print(response["answer"])

RuntimeError: Error loading essay.txt

In [47]:
pip install langchain-mistralai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 2.6 MB/s eta 0:00:00
  Attempting uninstall: httpx
    Found existing installation: httpx 0.27.0
    Uninstalling httpx-0.27.0:
      Successfully uninstalled httpx-0.27.0
Note: you may need to restart the kernel to use updated packages.


In [22]:
# Load the tokenizer associated with the specified model
#tokenizer = AutoTokenizer.from_pretrained(llm_model, padding=True, truncation=True, max_length=512)
# Define a question-answering pipeline using the model and tokenizer
question_answerer = pipeline(
    "question-answering", 
    model=model, 
    tokenizer=tokenizer,
    return_tensors='pt'
)

# Create an instance of the HuggingFacePipeline, which wraps the question-answering pipeline
# with additional model-specific arguments (temperature and max_length)
llm_generation = HuggingFacePipeline(
    pipeline=question_answerer,
    model_kwargs={"temperature": 0.7, "max_length": 512},
)



# Create a retriever object from the 'db' with a search configuration where it retrieves up to 4 relevant splits/documents.
retriever = KNOWLEDGE_VECTOR_DATABASE.as_retriever(search_kwargs={"k": 4})

# Create a question-answering instance (qa) using the RetrievalQA class.
# It's configured with a language model (llm), a chain type "refine," the retriever we created, and an option to not return source documents.
qa = RetrievalQA.from_chain_type(llm=llm_generation, chain_type="refine", retriever=retriever, return_source_documents=False)

The model 'BertLMHeadModel' is not supported for question-answering. Supported models are ['AlbertForQuestionAnswering', 'BartForQuestionAnswering', 'BertForQuestionAnswering', 'BigBirdForQuestionAnswering', 'BigBirdPegasusForQuestionAnswering', 'BloomForQuestionAnswering', 'CamembertForQuestionAnswering', 'CanineForQuestionAnswering', 'ConvBertForQuestionAnswering', 'Data2VecTextForQuestionAnswering', 'DebertaForQuestionAnswering', 'DebertaV2ForQuestionAnswering', 'DistilBertForQuestionAnswering', 'ElectraForQuestionAnswering', 'ErnieForQuestionAnswering', 'ErnieMForQuestionAnswering', 'FalconForQuestionAnswering', 'FlaubertForQuestionAnsweringSimple', 'FNetForQuestionAnswering', 'FunnelForQuestionAnswering', 'GPT2ForQuestionAnswering', 'GPTNeoForQuestionAnswering', 'GPTNeoXForQuestionAnswering', 'GPTJForQuestionAnswering', 'IBertForQuestionAnswering', 'LayoutLMv2ForQuestionAnswering', 'LayoutLMv3ForQuestionAnswering', 'LEDForQuestionAnswering', 'LiltForQuestionAnswering', 'LlamaForQu

In [23]:
question = "Que pense valérie Hayer de la guerre à Gaza ?"
result = qa.result({"query": question})
print(result["result"])

ValueError: Context information is below. 
------------
contenu: J’ai voté en faveur de cette résolution relative aux attaques infâmes menées par le groupe terroriste du Hamas en Israël, ainsi qu’à la détérioration très préoccupante de la situation humanitaire dans la région. Je condamne avec la plus grande fermeté les attaques barbares menées par le Hamas contre Israël, ayant conduit à la mort de plus de 1400 Israéliens et de nombreuses prises d’otage. Face à ces attaques, notre discours doit être ferme, sans pudeur ni complaisance : le Hamas est un groupe
------------
Given the context information and not prior knowledge, answer the question: Que pense valérie Hayer de la guerre à Gaza ?
 argument needs to be of type (SquadExample, dict)

In [19]:
retriever = KNOWLEDGE_VECTOR_DATABASE.as_retriever()

In [20]:
docs = retriever.get_relevant_documents("Guerre à Gaza")

In [21]:
print(docs[0].page_content)

Tout en garantissant le droit légitime d’Israël à assurer sa défense et en nous positionnant pour l’élimination du Hamas, il est urgent d’agir pour venir en aide aux populations civiles de la bande de Gaza, alors que la situation humanitaire y est devenue extrêmement préoccupante. Grâce à Renew Europe, le Parlement européen s’est accordé sur la nécessité d’organiser une pause humanitaire, et d’augmenter urgemment l’aide humanitaire internationale à destination des populations civiles de la bande de Gaza.


In [18]:
question = "Que pense valérie Hayer de la guerre à Gaza ?"
result = qa.invoke({"query": question})
print(result["result"])

ValueError: Context information is below. 
------------
contenu: J’ai voté en faveur de cette résolution relative aux attaques infâmes menées par le groupe terroriste du Hamas en Israël, ainsi qu’à la détérioration très préoccupante de la situation humanitaire dans la région. Je condamne avec la plus grande fermeté les attaques barbares menées par le Hamas contre Israël, ayant conduit à la mort de plus de 1400 Israéliens et de nombreuses prises d’otage. Face à ces attaques, notre discours doit être ferme, sans pudeur ni complaisance : le Hamas est un groupe
------------
Given the context information and not prior knowledge, answer the question: Que pense valérie Hayer de la guerre à Gaza ?
 argument needs to be of type (SquadExample, dict)

In [5]:
# Specify the model name you want to use
model_name = "Intel/dynamic_tinybert"

# Load the tokenizer associated with the specified model
tokenizer = AutoTokenizer.from_pretrained(model_name, padding=True, truncation=True, max_length=512)

# Define a question-answering pipeline using the model and tokenizer
question_answerer = pipeline(
    "question-answering", 
    model=model_name, 
    tokenizer=tokenizer,
    return_tensors='pt'
)

# Create an instance of the HuggingFacePipeline, which wraps the question-answering pipeline
# with additional model-specific arguments (temperature and max_length)
llm = HuggingFacePipeline(
    pipeline=question_answerer,
    model_kwargs={"temperature": 0.7, "max_length": 512},
)

In [15]:
llm_generation("Que pense Valérie Hayer de la guerre à Gaza")

/Users/joevincentgaltie/Documents/valeRAGhayer/valeRAG/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


ValueError: Que pense Valérie Hayer de la guerre à Gaza argument needs to be of type (SquadExample, dict)